### Perform Necessary Installations

In [ ]:
!pip install bs4
!pip install requests
!pip install pandas
!pip numpy

In [2]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import re
import numpy as np

### Define Helper Functions
We will use those throughout our notebook as they make our code resuable, extensible, and readable 

In [3]:
def flatten_comprehension(matrix):
     return [item for row in matrix for item in row]


def extract_rows_from_tables(tables):
    rows = []
# Now, iterate over each table in the ResultSet
    for table in tables:
        # For each table, you can now call find_all on it
        rows.append(table.find_all('tr'))
        # Do something with rows, e.g., print them
        
    
    return rows

def extract_raw_data(column_data,recurse_thru_a = False, find_tags = 'td', extract_provider_links = False, pattern =''):
    raw_data = []
    for row in column_data:
        row_data = row.find_all(find_tags)
        individual_row_data = []
        
        if(recurse_thru_a == True):
            extracted_links = [data.find('a')['href'] if data.find('a') else data.text.strip() for data in row_data]
#             print("extracted links: ", extracted_links)
            link_to_page = extracted_links[0] if extracted_links else None
#             print(f'Link to page: {link_to_page}')
            individual_row_data.append(link_to_page)
        # print("row_data[0] = ", row_data[0])
        individual_row_data +=[data.text.strip() for data in row_data]
        
        if extract_provider_links == True:
            extracted_links = [data.find('a')['href'] if data.find('a') else data.text.strip() for data in row_data]
            for element in extracted_links:
                matches = re.findall(pattern, element)
                if(matches):
                    individual_row_data.append(element)

        raw_data.append(individual_row_data)
    return raw_data
def extract_providers_link(column_data, find_tags = 'td',  pattern = ''):
    raw_data = []
    for row in column_data:
        row_data = row.find_all(find_tags)
        individual_row_data = []
        
       
        extracted_links = [data.find('a')['href'] if data.find('a') else data.text.strip() for data in row_data]
        
        
#         print(extracted_links)
        for element in extracted_links:
            matches = re.findall(pattern, element)
            if(matches):
                # print(f"'{element}' contains {len(matches)} occurrence(s) of {pattern}")
                individual_row_data +=[data.text.strip() for data in row_data]
                individual_row_data.append(element)
                raw_data.append(individual_row_data)

                
#         individual_row_data +=[data.text.strip() for data in row_data]
#         print(f'{individual_row_data}')
    return raw_data
def append_data_to_df(df, raw_data):
    for data in raw_data:
#         print(f'data: {data}')
        length = len(df)
        df.loc[length] = data
    return df

def preprocess_extracted_sats(cont_sats_extracted, row_names, attr_index = 0):
    for index, sat in enumerate(cont_sats_extracted):
#         print("len(sat) = > ",len(sat))
#         print("len(row_names) = > ",len(sat))
        
        if len(sat) <= len(row_names) - 1:
#             print(f'found a channel "{sat}"')
            sat_dir = cont_sats_extracted[index - 1][attr_index]
#             print(f'New Attr "{sat_dir}"')
            sat.insert(attr_index, sat_dir)
            cont_sats_extracted[index] = sat
#             print(f'New channel "{cont_sats_extracted[index]}"')

    return cont_sats_extracted

def append_region_to_extracted_sats(cont_sats_extracted, continent):
    for index, sat in enumerate(cont_sats_extracted):
        cont_sats_extracted[index] =  sat + [continent]
    return cont_sats_extracted

In [4]:
def extract_raw_data_alt(column_data,recurse_thru_a = False, find_tags = 'td', extract_provider_links = False, pattern =''):
    raw_data = []
    for row in column_data:
        row_data = row.find_all(find_tags)
        columns = row_data
        individual_row_data = []
        
        
        if(recurse_thru_a == True):
            extracted_links = [data.find('a')['href'] if data.find('a') else data.text.strip() for data in row_data]
            link_to_page = extracted_links[0] if extracted_links else None
            individual_row_data.append(link_to_page)

        individual_row_data +=[data.text.strip() for data in row_data]
        
       
        # print(len(columns))
        if len(columns) == 10:
            font = columns[0].find_all('font')[0]
            # print('font: ', font)
            
            br = font.find_all('br')
            # print('br: ', br)
            freq = None
            beam = None
            eirp = None
            for i, thing in enumerate(br):
                if i == 0:
                    freq = thing.previous_sibling.get_text(strip=True)
                elif i == 1:
                    beam = thing.previous_sibling.get_text(strip=True)
                else:
                    eirp = thing.next_sibling.get_text(strip=True) if thing.next_sibling else None
            # print(f'freq: {freq}, beam: {beam}, eirp: {eirp}')
        individual_row_data.append(freq)
        individual_row_data.append(beam)
        individual_row_data.append(eirp)
        
        if len(columns) == 10:
            fonts = columns[8].find_all('font')
            # print('fonts for network: ', fonts)
            fonts = [bs(str(data).replace('<br/>',',')).text for data in fonts]
            first_font = fonts[0] if fonts else None
            individual_row_data.append(first_font)
        else:
            fonts = columns[6].find_all('font')
            fonts = [bs(str(data).replace('<br/>',',')).text for data in fonts]
            first_font = fonts[0] if fonts else None
            individual_row_data.append(first_font)
            # individual_row_data += fonts.pop()
        # print('fonts: ', fonts)
                        
        # individual_row_data +=[data.text.strip() for data in row_data]
        
        # individual_row_data +=[BeautifulSoup(str(data).replace('<br/>',',')).text for data in row_data]
        
        if extract_provider_links == True:
            extracted_links = [data.find('a')['href'] if data.find('a') else data.text.strip() for data in row_data]
            for element in extracted_links:
                matches = re.findall(pattern, element)
                if(matches):
                    individual_row_data.append(element)

        raw_data.append(individual_row_data)
    return raw_data

In [5]:
def max_char_count(df_column):
    lengths = df_column.str.len()
    # Find the index of the longest string
    index_of_max = lengths.idxmax()
    print(index_of_max, ', ',df_column.iloc[index_of_max])
    return df_column.str.len().max()

def display_max_length(df):
    for i, col in enumerate(df.keys()):
        char_count = max_char_count(df[col].astype(str))
        print(f'COLUMN: {col},  MAX_LENGTH: {char_count}')


def match_first_part(string):
    match = re.search(r'^[^(]+', string)
    if match:
        return match.group()
    else:
        return string
        

## Extract Satellites for All Regions

### List Continents and concatenate with their url

In [518]:
continents = ['asia', 'europe', 'atlantic', 'america']
baseurl = 'https://www.lyngsat.com/'

# Construct the URLs
endpoints = []
for c in continents:
    endpoints.append(f'{baseurl}{c}.html')
endpoints

['https://www.lyngsat.com/asia.html',
 'https://www.lyngsat.com/europe.html',
 'https://www.lyngsat.com/atlantic.html',
 'https://www.lyngsat.com/america.html']

### Scrape each Endpoint

In [519]:
satellites = []
for index, c in enumerate(continents):
    page = requests.get(endpoints[index])
    soup = bs(page.text, 'html')
    
    
    satellites.append({f'{c}':soup})

In [ ]:
row_names = ['Position','Name','Frequency','Launch_Date']
cont_sats_df = pd.DataFrame(columns = row_names)
# 
for index, cont in enumerate(continents):
#     print('continent: ',cont)
    cont_sats_raw = satellites[index][cont]
    table = cont_sats_raw.find_all('table')[11]
    
    column_data = table.find_all('tr')
    
    cont_sats_extracted = extract_raw_data(column_data)
    print(cont_sats_extracted[1:])
    # cont_sats_extracted = preprocess_extracted_sats(cont_sats_extracted, row_names)
    # cont_sats_extracted = append_region_to_extracted_sats(cont_sats_extracted, cont)    
    # print((cont_sats_extracted[0][1]))
    # cont_sats_df.loc[len(cont_sats_df)] = 
    temp_df = pd.DataFrame(cont_sats_extracted[1:], columns=row_names)
    cont_sats_df = pd.concat([cont_sats_df, temp_df], ignore_index=True)
    # cont_sats_df = append_data_to_df(cont_sats_df, cont_sats_extracted[1:])
    
# Switch the first (position) with the second column (name)
cont_sats_df = cont_sats_df.iloc[:, [1, 0] + list(range(2, len(cont_sats_df.columns)))]
cont_sats_df


In [563]:
cont_sats_df

,Name,Position,Frequency,Launch_Date
0,NSS 9,177.0°W,C,190104
1,Intelsat 18,180.0°E,CKu,231124
2,Eutelsat 172B,172.0°E,CKu,240306
3,Horizons 3e,169.0°E,Ku,240302
4,Intelsat 19,166.0°E,CKu,240401
...,...,...,...,...
223,Galaxy 33,133.0°W,CKu,230926
224,SES 22,135.0°W,C,240302
225,SES 19,135.0°W,,
226,AMC 6,139.0°W,C,210906


### Get Launching Rockets Details

Now, since we have the satellites, we just need to get their launching rockets & dates into a dataframe, and then merge those two

In [565]:
continents = ['asia', 'europe', 'atlantic', 'america']
baseurl = 'https://www.lyngsat.com/tracker/'

# Construct the URLs
endpoints = []
for c in continents:
    endpoints.append(f'{baseurl}{c}.html')
endpoints

['https://www.lyngsat.com/tracker/asia.html',
 'https://www.lyngsat.com/tracker/europe.html',
 'https://www.lyngsat.com/tracker/atlantic.html',
 'https://www.lyngsat.com/tracker/america.html']

In [566]:
sat_rockets = []
for index, c in enumerate(continents):
    page = requests.get(endpoints[index])
    soup = bs(page.text, 'html')    
    
    sat_rockets.append({f'{c}':soup})

In [ ]:
sat_rockets

#### Get the launching URL for each satellite

In [564]:


row_names = ['URL','Position','Name','Frequency', 'Launch Date', 'Region']
cont_sats_df = pd.DataFrame(columns=row_names)

# index = 0
# cont = 'asia'
for index, cont in enumerate(continents):

    cont_sats_rockets = sat_rockets[index][cont]
    # cont_sats_rockets = sat_rockets[2]['atlantic']

    # print(cont_sats_rockets)

    table = cont_sats_rockets.find_all('table')[11]
    #     print(table)

    column_data = table.find_all('tr')
    #     print(column_data)

    cont_sats_extracted = extract_raw_data(column_data, True)

    # cont_sats_extracted = preprocess_extracted_sats(cont_sats_extracted, row_names)
    cont_sats_extracted = append_region_to_extracted_sats(cont_sats_extracted, cont)    

    # print(len(cont_sats_extracted))


    cont_sats_df = append_data_to_df(cont_sats_df, cont_sats_extracted[1:])
    # cont_sats_df =  cont_sats_extracted
        
    
# Switch the first (position) with the second column (name)
cont_sats_df = cont_sats_df.iloc[:, [1, 0] + list(range(2, len(cont_sats_df.columns)))]
cont_sats_df

,Position,URL,Name,Frequency,Launch Date,Region
0,167.0°W,https://www.lyngsat.com/tracker/TDRS-8.html,TDRS 8 (incl. 11.4°),,240319,asia
1,171.0°W,https://www.lyngsat.com/tracker/TDRS-10.html,TDRS 10 (incl. 9.4°),,240319,asia
2,174.0°W,https://www.lyngsat.com/tracker/TDRS-11.html,TDRS 11 (incl. 2.7°),,240320,asia
3,177.0°W,https://www.lyngsat.com/tracker/NSS-9.html,NSS 9,C,240319,asia
4,177.0°W,https://www.lyngsat.com/tracker/Yamal-300K.html,Yamal 300K,,240319,asia
...,...,...,...,...,...,...
308,135.0°W,https://www.lyngsat.com/tracker/SES-19.html,SES 19,,240319,america
309,138.9°W,https://www.lyngsat.com/tracker/Spaceway-2.html,Spaceway 2 (incl. 3.4°),,240319,america
310,139.0°W,https://www.lyngsat.com/tracker/AMC-6.html,AMC 6,C,240319,america
311,139.2°W,https://www.lyngsat.com/tracker/Eutelsat-139-W...,Eutelsat 139 West A (incl. 3.7°),,240319,america


In [ ]:
cont_sats_df['URL']

#### Get The Data
for each satellite URL obtained above

In [ ]:
rocket_list = []
rocket_date = []
for index, cont in enumerate(cont_sats_df['URL']):
    url = cont_sats_df['URL'][index]
    page = requests.get(url)
    soup = bs(page.text, 'html')
#     soup.find_all('font')[13]

    entry  = soup.find_all('font')
    
    entry = ' '.join(map(str,entry))
    sat_name = cont_sats_df['Name'][index]

    x = re.search("launched with (.+) \d\d\d\d", entry)
    
    _date = re.search('\d{4}-\d{2}-\d{2}', entry)
    if _date:
        rocket_date.append(
            {
            f'{sat_name}': _date.group()
        })
        # return match.group()
    else:
        print(f'Couldnt find a launching DATE for {sat_name} with entry {entry}')
#     print(x.group(1))
    
    
    
    print(f'Satellite name: {sat_name}')
    if(x):
#         print(f'Found l')
        rocket_list.append(
            {
            f'{sat_name}': x.group(1)
        })
    else:
        print(f'Couldnt find a launching rocket for {sat_name} with entry {entry}')
        
    print(f'Processed {index + 1} Satellites out of {len(cont_sats_df.index)}')
    

In [ ]:
rocket_list

In [ ]:
rocket_date

In [ ]:
rocket_list


list_of_lists = [list(d.items())[0] for d in rocket_list]
# new_list
row_names = ['Sat_name', 'L_Rocket']

list_of_lists_date = [list(d.items())[0] for d in rocket_date]

rockets_date_df = pd.DataFrame(list_of_lists_date, columns=['Sat_name', 'L_Date'])

rockets_df = pd.DataFrame(list_of_lists,columns = row_names)


## Merge both dataframes, creating the Satellite Launches CSV
rockets_df = pd.merge(left=rockets_df, right=rockets_date_df, left_on='Sat_name', right_on='Sat_name')

rockets_df.to_csv('C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/v6_Satellites_Rockets.csv', index = False)

In [ ]:

# The original sattelite CSV; missing launches
sat_df = pd.read_csv('E:/AUC/23-24/Spring/Database/Project/CSV Files/Satellites.csv')
# rockets_df = pd.read_csv('E:/AUC/23-24/Spring/Database/Project/CSV Files/Satellites_Rockets.csv')

# The Sattelite Launches CSV; missing rest of satellite data
rockets_df = pd.read_csv('C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/v6_Satellites_Rockets.csv')

rockets_df = rockets_df[['Sat_name','L_Date']]


merged_df = pd.read_csv('E:/AUC/23-24/Spring/Database/Project/CSV Files/Satellites_Rockets.csv')

# Merge both
final_df = pd.merge(left=merged_df, right=rockets_df, left_on='Name', right_on='Sat_name', how='left')

final_df.shape
# merged_df.shape

In [ ]:
# Remove redundant and irrelevant columns
final_df.drop(columns=['Sat_name', 'Frequency','Launch_Date'], inplace=True)

In [ ]:
# Populate missing date values
final_df['L_Date'].fillna('1900-01-01',inplace=True)

In [ ]:
# Save
final_df.to_csv('C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/v6_final_satellites.csv', index=False)

## Scrape Channels and Providers
For each satellite, we:
1. Scrape the Channels/ Providers records, treating them as one.
    a. Identify the Providers and merge with the original dataframe
    b. Save that
2. Assign Providers to Channels
3. Clean the Dataframes
    a. Split the System/SR/FEC columns
    b. Extract the languages
    c. Extract the Frequency

We begin by defining *helper functions* & *attributes*

In [ ]:
# Retrieves data tables from a given url
def get_table(url):
    
    page = requests.get(url)
    soup = bs(page.text, 'html')

    table = soup.find_all('table',{'border':"", 'cellpadding':"0", 'cellspacing':"0", 'width':"720"} )

    return table


In [569]:
baseurl = 'https://www.lyngsat.com/'
# Contains the names of our Satellites
sats_rockets = pd.read_csv('E:/AUC/23-24/Spring/Database/Project/CSV Files/Satellites_Rockets.csv')

Get Satellite Safe names

In [570]:

# Prepare satellite names for incorporation into URLs

raw_sat_names = [link.replace(" ", "-") for link in sats_rockets['Name']]
raw_sat_names = [link.replace("'", "" ) for link in raw_sat_names]
raw_sat_names = [link.replace("ü", "u" ) for link in raw_sat_names]
raw_sat_names = [link.replace("/", "-" ) for link in raw_sat_names]
raw_sat_names = [link.replace("Ä", "A" ) for link in raw_sat_names]

sat_url_names = [re.split('-\(',link)[0] for link in raw_sat_names]
# baseurl

<>:9: SyntaxWarning: invalid escape sequence '\('
<>:9: SyntaxWarning: invalid escape sequence '\('
C:\Users\Ali\AppData\Local\Temp\ipykernel_2736\410144265.py:9: SyntaxWarning: invalid escape sequence '\('
  sat_url_names = [re.split('-\(',link)[0] for link in raw_sat_names]


In [ ]:
sat_url_names

We Define a dictionary to map between the original satellite name and its safe name

The safe name allows us to store the sattelite name safely on Windows in the filename

*This is also useful later when mapping back from the safe name to the original sat. name before import data to mysql*

In [ ]:
very_raw_sat_urls = merged_df['Name']

sat_names_dict = dict(zip(sat_url_names, very_raw_sat_urls))
sat_names_dict

def get_normal_sat_name(safe_sat_name):
    return sat_names_dict.get(safe_sat_name)

### Scrape Channels & Providers

Since our focus is on Network Providers & Channels (TV, Radio), we exclude packages from our dataframes

In [ ]:
## ALT

## SCRAPE CHANNELS + PROVIDERS
# index = 0
# sat_name = 'NSS-9'
for index, sat_name in enumerate(sat_url_names):
    
    url = f'{baseurl}{sat_name}.html'
    print(f'{index + 1} Processing satellite {sat_name} with url {url}')

    # Scrape endpoint, retrieving all data tables 
    sat_table = get_table(url)

    # Retrieve all tr's from the returned tables
    # Returns a list of lists (/table)
    column_data = extract_rows_from_tables(sat_table)

    for i, table in enumerate(column_data):
    #     print("table#",i)
        if(len(table) > 2):
            _temp = column_data[i]
            _temp = _temp[2:len(_temp)-1]
        #     print(_temp)
            column_data[i] = _temp

    # Flattens the list. Now, we have a list of tr tags
    column_data = flatten_comprehension(column_data)

    # Clean and extract the data values from the tags
    channels_extracted = extract_raw_data_alt(column_data)



    networks_extracted = extract_providers_link(column_data, pattern='.*providers')



    # Define the columns for our Main Dataframe
    row_names = ['Freq/beam','SR/FEC', 'SID', 'Provider/Channel','undef','Compression','VPID','Audio', 'Encryption', 'Src_Updated', 'system_alt', 'sr_alt', 'fec_alt', 'freq_alt', 'beam_alt', 'eirp_alt','encryption_alt']

    # Define the columns for our Networks Dataframe
    netw_row_names = ['Freq/beam','SR/FEC', 'SID', 'Provider/Channel','undef','Compression','VPID','Audio', 'Encryption', 'Src_Updated', 'Provider_URL']

    # Construct the Network Dataframe
    netw_df = pd.DataFrame(networks_extracted, columns = netw_row_names)

    # Add Frequency/Beam to our channels (as inherited from the preceeding element)
    channels_extracted = preprocess_extracted_sats(channels_extracted, row_names)

    # Add System/SR/FEC to our channels (as inherited from the preceeding element)
    channels_extracted = preprocess_extracted_sats(channels_extracted, row_names, 1)

    # Construct the Main Dataframe
    sat_df = pd.DataFrame(channels_extracted, columns = row_names)
 
    sat_df[['system_alt', 'sr_alt', 'fec_alt']] = sat_df[['system_alt', 'sr_alt', 'fec_alt']].fillna(method='ffill')    
    # merged_sat_df['Frequency'] = merged_sat_df['Frequency'].fillna(method='ffill')


    # Refine the Networks Dataframe to remove clutter/ redundant attributes
    netw_df = netw_df[['Provider/Channel', 'Provider_URL']]

    # Merge the Main and Network Dataframes
    merged_df = pd.merge(left=sat_df, right=netw_df, how='outer',left_on='Provider/Channel', right_on='Provider/Channel')

    # Adjust the filename to be safe (for saving the file on Windows)
    safe_sat_name = sat_name.replace('/', '-') # Replace '/' with '_'

    try:
        chans_links = extract_providers_link(column_data,pattern='.*tvchannels|.*radiochannels')
        # print(chans_links)
        chan_links_df = pd.DataFrame(data=chans_links)
        chan_links_df = chan_links_df.iloc[:, [1,8]]
        chan_links_df.rename(columns ={1: "Channel", 8: "Channel_URL"}, inplace=True)
        merged_df = pd.merge(left=merged_df, right=chan_links_df, left_on='Provider/Channel', right_on='Channel',how='outer')
    except:
        print(f'Problems with satellite: {safe_sat_name}')
    
    try:
        pkgs_extracted = extract_providers_link(column_data, pattern='.*packages')
        if(len(pkgs_extracted) > 0):
            pkgs_extracted_df = pd.DataFrame(data=pkgs_extracted)
            pkgs_extracted_df= pkgs_extracted_df.iloc[:, [3,21]]
            pkgs_extracted_df.rename(columns ={3: "Pkg_Name", 21: "Pkg_URL"}, inplace=True)
            merged_df = pd.merge(left=merged_df, right=pkgs_extracted_df, left_on='Provider/Channel', right_on='Pkg_Name',how='outer')

        # print(chan_links_df.shape)
        # merged_df.pd_merge(left=merged_df, right )

    except: 
        print(f'problem with pkgs extraction')
    # Save the merged Dataframe
    merged_df.to_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/providers/v6/{index + 1}_{safe_sat_name}_channels.csv',index=False)



#### Assign Providers to Channels

In [ ]:

for index, sat_name in enumerate(sat_url_names):
#     index = 1
#     sat_name = 'Intelsat-18'
    safe_sat_name = sat_name.replace('/', '-') # Replace '/' with '_'
    my_df = pd.read_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/providers/v6/{index + 1}_{safe_sat_name}_channels.csv')

   
    print(f'{index + 1} Processing sat: {safe_sat_name}')

    my_df['is_Provider'] = np.where(my_df['Provider_URL'].astype(str).str.contains('http', regex=True, na=False), True, False)

    providers_df = my_df[my_df['is_Provider'] == True]
    channels_df = my_df[my_df['is_Provider'] == False]

    my_df['Provider'] = None

    for i in range(len(my_df.index)):
        isProvider = my_df.loc[i, 'is_Provider']
    
        if isProvider == True:
            my_df.loc[i, 'Provider'] = my_df.loc[i, 'Provider/Channel']
            continue

        if i > 0:
            my_df.loc[i, 'Provider'] = my_df.loc[i-1, 'Provider'] 
    my_df.to_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/providers/v6/assigned/{index + 1}_{safe_sat_name}_channels.csv',index=False)

# my_df

## Clean our Dataframes

##### Define Helper Functions

In [572]:
# Function to extract string until the last capital character
def extract_until_last_capital(s):
    match = re.search(r'(.*[A-Z])', s)
    if match:
        return match.group(1)
    return s
def find_pattern_and_join(entry):
    matches = re.findall(r"([A-Z][a-z]+)", entry)
    return ' '.join(matches) # Join the matches into a single string
def split_beam_eirp(entry, beam_eirp_index = 0):
    temp = re.split(r'(?<=\d)(?=\D)', entry)
    if beam_eirp_index == 1:
        # Check if the value is not None
        if(len(temp) > 1):
            return temp[beam_eirp_index] 
        else:
            'None'
    else:
        return temp[beam_eirp_index]

In [ ]:
skipped_sats_url = []

#### Perform The Cleaning

In [ ]:
# We drop irrelevant columns and packages

for index, sat_name in enumerate(sat_url_names):
    safe_sat_name = sat_name.replace('/', '-') # Replace '/' with '_'
    hor_df = pd.read_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/providers/v6/assigned/{index + 1}_{safe_sat_name}_channels.csv')

    print(f'{index + 1} Processing sat: {safe_sat_name}')

    if len(hor_df.index) == 0:
        url = f'{baseurl}{sat_name}.html'
        skipped_sats_url.append(url)
        hor_df.to_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/providers/v6/assigned/cleaned/{index+1}_{safe_sat_name}_channels.csv', index=False)
        continue
    
    # Drop irrelevant columns
    hor_df.drop(columns=['undef','Src_Updated', 'Freq/beam', 'Encryption'], inplace = True)  
    
    # try:
    #     hor_df['FEC'] = hor_df['SR/FEC'].str[-3:]
    #     hor_df['SR'] = hor_df['SR/FEC'].str[-8:-3].str.extract('(\d+)')
    #     hor_df['SYSTEM'] = hor_df['SR/FEC'].str[0:6]
    # except:
    #     print("PROBLEM YO")

    # Drop the now-old composite column
    hor_df.drop('SR/FEC', axis=1, inplace=True)

    # Extract Languages
    hor_df['Audio'] = hor_df['Audio'].astype(str)

    hor_df['Languages'] = hor_df['Audio'].apply(find_pattern_and_join)
    
    hor_df.drop('Audio', axis=1, inplace=True)
    
    # if 'A' in df.columns:
    if 'Pkg_URL' in hor_df.columns:
        # Drop Packages
        hor_df.drop(hor_df[hor_df['Pkg_URL'].notna()].index, inplace=True)

    
    hor_df.to_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/providers/v6/assigned/cleaned/{index+1}_{safe_sat_name}_channels.csv', index=False)    

In [ ]:
# View the URLs that were skipped from cleaning due to lack of data
skipped_sats_url

In [ ]:
# View the current output for a cleaned dataframe
index = 1
safe_sat_name = 'Intelsat-18'

my_df = pd.read_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/providers/v5/assigned/cleaned/{index+1}_{safe_sat_name}_channels.csv')


In [ ]:
my_df.head()

## Get Country for Provider

Define Helper functions

In [ ]:


# Retrieves data tables from a given url
def get_country(url):
    
    page = requests.get(url)
    soup = bs(page.text, 'html')

    table = soup.find_all('table',{'width':"700"} )

    return table
def extract_country(url):
    my_pattern = "/\D\D/"
    url = str(url)
    # print('url => ', url)
    matches = re.search(my_pattern, url)
    if matches:
        country = matches.group() # Assuming the country is the first captured group
        country = country[1:-1]
        return country
    else:
        return None # or any default value you prefer



In [ ]:
new_df = pd.DataFrame(columns = ['Sattelite', 'Provider', 'Country'])


In [ ]:

## Extract Channel Country
x_df = None

netw_flag = True
for index, sat_name in enumerate(sat_url_names):
    
    safe_sat_name = sat_name.replace('/', '-') # Replace '/' with '_'
    new_col_name = 'Provider_Country'
    ref_col = 'Provider_URL'
    dir_name = 'prov_country'
    if netw_flag == False:
        new_col_name = 'Channel_Country'
        ref_col = 'Channel_URL'
        dir_name = 'chan_country'
        
        # May cause problems; remember to change the chan_country dir if cleaning for providers by setting netw_flag
    hor_df = pd.read_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/providers/v6/assigned/cleaned/chan_country/{index + 1}_{safe_sat_name}_channels.csv')
    
    # hor_df['Channel_Country'] = None
    print(f'{index + 1} Processing sat: {safe_sat_name}')

    if len(hor_df.index) == 0:
        # url = f'{baseurl}{sat_name}.html'
        # skipped_sats_url.append(url)
        print(f'Skipping sattelite: {safe_sat_name}')
        hor_df.to_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/providers/v6/assigned/cleaned/{dir_name}/{index+1}_{safe_sat_name}_channels.csv', index=False)
        continue    
    
    try:
        hor_df[new_col_name] = hor_df[ref_col].apply(extract_country)
        hor_df.to_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/providers/v6/assigned/cleaned/{dir_name}/{index +1}_{safe_sat_name}_channels.csv', index=False)
    except:
        print('problem yo')
        hor_df.to_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/providers/v6/assigned/cleaned/{dir_name}/{index +1}_{safe_sat_name}_channels.csv', index=False)



In [ ]:

new_df = pd.DataFrame(columns=['Satellite', 'Provider', 'Country'])

prov_urls = my_df['Provider_URL'].dropna().unique()

# prov_urls = ['https://www.lyngsat.com/tvchannels/us/AFN-Prime-Atlantic.html']
for url in prov_urls:
    table = get_country(url)
    column_data = extract_rows_from_tables(table)

    # Flattens the list. Now, we have a list of tr tags
    column_data = flatten_comprehension(column_data)
    col = extract_raw_data(column_data)
    if(len(col) > 1):
        provider = my_df[my_df['Provider_URL'] == url].iloc[0]['Provider/Channel']
        country = col[0][0].split('\n')[-1]
        new_df.loc[len(new_df)] = [safe_sat_name, provider ,country]

In [ ]:
new_df = pd.read_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/providers/v6/assigned/cleaned/prov_country/1_NSS-9_channels.csv' )

# new_df['Sat_Name'] = 'ali'


In [ ]:
all_channels = pd.DataFrame(columns=new_df.columns)
dfs_list = []

## Concatenate Satellites dataframes
Here, we concatenate channel instances alongside network providers, which will be the source of construction of our entities 

In [ ]:
# all_channels
for index, sat_name in enumerate(sat_url_names):
    print(sat_name)
    safe_sat_name = sat_name.replace('/', '-') # Replace '/' with '_'
    curr_df = pd.read_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/providers/v6/assigned/cleaned/prov_country/{index + 1}_{sat_name}_channels.csv')
    curr_df['Sattelite_Name'] = safe_sat_name
    # all_channels = pd.concat([all_channels, curr_df])
    dfs_list.append(curr_df)


In [ ]:
all_channels = pd.concat(dfs_list, ignore_index=True)
# Remove duplicates
all_channels.drop_duplicates(inplace=True)
all_channels.shape

In [ ]:
# all_channels.drop(columns=['Src_Updated', 'Encryption', 'Audio', 'undef','SR/FEC', 'Freq/beam'], inplace=True)

all_channels.to_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/all_channels_raw_v6.csv',index=False)

In [ ]:
all_channels = pd.read_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/all_channels_raw_v6.csv')

all_channels.shape

In [ ]:
# Remove entries that have '(feeds)' as their Provider/Channel name as they are not channels or providers 
feeds = all_channels[all_channels['Provider/Channel'].astype(str).str.contains('^\(f.*\)|^\[')]

indices = feeds.index.values.tolist()  

all_channels.drop(axis = 0, index = indices, inplace=True)

In [ ]:
# print(temp_all.shape)
print(all_channels.shape)

In [ ]:
# The same with channels/ provs that start with '@'
indices = all_channels[all_channels['Provider/Channel'].astype(str).str.contains('^@')].index.values.tolist()

In [ ]:
all_channels.drop(axis=0, index=indices, inplace=True)

Get Channel Instances

In [ ]:
# channel_instances_df = all_channels[~all_channels['Channel_URL'].isna()]

channel_instances_df = all_channels[all_channels['is_Provider'] == False]

# Retrieve records with non-empty channel names
channel_instances_df = channel_instances_df[channel_instances_df['Provider/Channel'].notna()]

channel_instances_df.shape

Obtain Unique Channels

In [ ]:

unique_channels = channel_instances_df.drop_duplicates(subset=['Provider/Channel'])
# unique_channels = unique_channels[unique_channels['Provider/Channel'].notna()]

unique_channels.shape
# df2 = group.apply(lambda x: x['Channel_Country'].unique())

In [ ]:
entity_channels_df = unique_channels[['Provider/Channel','Channel_Country']]
entity_channels_df.to_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/entities/v6_channels.csv',index=False)
entity_channels_df.shape

Construct Network Providers Entity

In [ ]:
networks_df = all_channels[all_channels['is_Provider'] == True]
networks_df.shape

In [ ]:
unique_networks = networks_df.drop_duplicates(subset=['Provider'])

unique_networks.shape

In [ ]:
entity_networks_df = unique_networks[['Provider', 'Provider_Country']]

entity_networks_df

In [ ]:
entity_networks_df.to_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/entities/v6_networks.csv',index=False)

Construct Channel Instance Languages Entity

In [ ]:
# channel_instances_df = channel_instances_df[channel_instances_df['Provider/Channel'].notna()]
channel_instances_language = channel_instances_df[['Provider/Channel','Sattelite_Name','freq_alt', 'Languages']]

# Split by each space
##  The result is a DataFrame where each value in the 'Languages' column is a list of words.
channel_instances_language['Languages'] = channel_instances_language['Languages'].str.split(' ')

# transform the df by expanding the 'Languages' column
## Result: one language per row (atomicity)
channel_instances_language = channel_instances_language.explode('Languages')

In [ ]:
channel_instances_language.to_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/entities/v6_channels_instance_language.csv',index=False)


In [ ]:
channel_instances_df.keys()

Construct the Channel Instance Encryption entity

In [ ]:
channel_instances_encryption = channel_instances_df[['Provider/Channel','Sattelite_Name','freq_alt', 'encryption_alt']]
# channel_instances_encryption['encryption_alt'].value_counts()

In [ ]:

# channel_instances_language = channel_instances_df[['Provider/Channel','Sattelite_Name','freq_alt', 'Languages']]
channel_instances_encryption['encryption_alt'] = channel_instances_encryption['encryption_alt'].str.split(',')

channel_instances_encryption = channel_instances_encryption.explode('encryption_alt')


In [ ]:

channel_instances_encryption

In [ ]:
channel_instances_encryption.to_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/entities/v6_channels_instance_encryption.csv',index=False)


In [ ]:
channel_instances_df = pd.read_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/entities/channel_instances.csv')
# print(channel_instances_df.keys())
# channel_instances_df.shape

# channel_instances_df[['Sattelite_Name','Provider/Channel','FEC']][101:150]


In [ ]:
channel_instances_df = channel_instances_df[channel_instances_df['is_Provider'] == False]

channel_instances_df.shape


In [ ]:
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# channel_instances_df[channel_instances_df['Sattelite_Name'] == 'Intelsat-18'][34:100]

channel_instances_df.drop_duplicates(subset=['Provider/Channel','Sattelite_Name', 'freq_alt'], inplace=True)
# channel_instances_df['Channel'].isna().sum()


In [ ]:
channel_instances_df.to_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/entities/v6_channel_instances.csv')

In [ ]:
# duplicates_df = channel_instances_df[channel_instances_df.duplicated(subset=['Channel','Sattelite_Name', 'freq_alt'], keep=False)]

# duplicates_df.shape

channel_instances_df['fec_alt'].value_counts()

In [ ]:
for col in duplicates_df.keys():
    # val_cnt = (duplicates_df[col].isna())
    print(f'{col}: {duplicates_df[col].isna().sum()}')
    
    

## Adjusting the Scraping Methodology

I found out that my beam, eirp, FEC,and SR attributes were wrong for some records. Hence, I went over it again

In [ ]:
def is_FEC(sus_txt):
    match = re.search('^\d/\d', sus_txt)
    if match:
        return True
    return False

In [ ]:
print(is_FEC('2/3'))   

In [ ]:
## You can see the difference between this version and the one in the very beginning of the notebook
def extract_raw_data_alt(column_data,recurse_thru_a = False, find_tags = 'td', extract_provider_links = False, pattern =''):
    raw_data = []
    for row in column_data:
        row_data = row.find_all(find_tags)
        columns = row_data
        individual_row_data = []
        
        
        if(recurse_thru_a == True):
            extracted_links = [data.find('a')['href'] if data.find('a') else data.text.strip() for data in row_data]
            link_to_page = extracted_links[0] if extracted_links else None
            individual_row_data.append(link_to_page)

        individual_row_data +=[data.text.strip() for data in row_data]
        
        # for data in row_data:
        font = columns[1].find_all('font')[0]
        # font = font.replace('<br/>', ',')
        br = font.find_all('br')
        
        system = None
        SR = None
        FEC = None
        
        len_br = len(list(enumerate(br)))
        # for i, thing in enumerate(br):
        for i, thing in enumerate(br):  
            # print(f'{i} -> font: {font}')
            # print(f'{i}: {thing}')    
            if i == 0:
                system = thing.previous_sibling.get_text(strip = True)
            if len_br == 3:
                if i == 1:
                    SR = thing.next_sibling.get_text(strip = True) if thing.next_sibling else None
                elif i == 2:
                    if thing.next_sibling:
                        fec_flag = is_FEC(thing.next_sibling.get_text(strip = True))
                        if fec_flag == True:
                            FEC = thing.next_sibling.get_text(strip = True)
                        else:
                            SR = thing.next_sibling.get_text(strip = True)
            elif len_br == 2:
                if thing.previous_sibling:
                    # print(f'prev: {thing.previous_sibling}')
                    SR = thing.previous_sibling.get_text(strip = True)
                if thing.next_sibling:
                    if(is_FEC(thing.next_sibling.get_text(strip = True))):
                        FEC = thing.next_sibling.get_text(strip = True)
                    else:
                        SR = thing.next_sibling.get_text(strip = True)
                    
        individual_row_data.append(system)
        individual_row_data.append(SR)
        individual_row_data.append(FEC)
                
        # print(f'system: {system}, SR: {SR}, FEC: {FEC}')
                
            # print(f'{i}: {thing.previous_sibling.get_text(strip=True)}')
        
            # if i == len(br):
            #     print('last element')

        
        # print(f'joined_text: {br}')
           
        # print(len(columns))
        if len(columns) == 10:
            font = columns[0].find_all('font')[0]
            # print('font: ', font)
            
            br = font.find_all('br')
            # print('br: ', br)
            freq = None
            beam = None
            eirp = None
            for i, thing in enumerate(br):
                if i == 0:
                    freq = thing.previous_sibling.get_text(strip=True)
                elif i == 1:
                    beam = thing.previous_sibling.get_text(strip=True)
                else:
                    eirp = thing.next_sibling.get_text(strip=True) if thing.next_sibling else None
            # print(f'freq: {freq}, beam: {beam}, eirp: {eirp}')
        individual_row_data.append(freq)
        individual_row_data.append(beam)
        individual_row_data.append(eirp)
        
        if len(columns) == 10:
            fonts = columns[8].find_all('font')
            # print('fonts for network: ', fonts)
            fonts = [bs(str(data).replace('<br/>',',')).text for data in fonts]
            first_font = fonts[0] if fonts else None
            individual_row_data.append(first_font)
        else:
            fonts = columns[6].find_all('font')
            fonts = [bs(str(data).replace('<br/>',',')).text for data in fonts]
            first_font = fonts[0] if fonts else None
            individual_row_data.append(first_font)
            # individual_row_data += fonts.pop()
        # print('fonts: ', fonts)
                        
        # individual_row_data +=[data.text.strip() for data in row_data]
        
        # individual_row_data +=[BeautifulSoup(str(data).replace('<br/>',',')).text for data in row_data]
        
        if extract_provider_links == True:
            extracted_links = [data.find('a')['href'] if data.find('a') else data.text.strip() for data in row_data]
            for element in extracted_links:
                matches = re.findall(pattern, element)
                if(matches):
                    individual_row_data.append(element)

        raw_data.append(individual_row_data)
    return raw_data

In [ ]:
# result
channels_df = pd.read_csv('E:/AUC/23-24/Spring/Database/Project/CSV Files/entities/v6_channels.csv')
networks_df = pd.read_csv('E:/AUC/23-24/Spring/Database/Project/CSV Files/entities/v6_networks.csv')

channels_df.shape

#### Sidenote
 we apply the match_first_part fn over and over to remove badly-named channels such as 'CHAN NAME (dir 2042- upd 492 -kd42 493299 0 slkd)'

 * We will use this method below multiple times on many of our entities to ensure consistency *


In [ ]:

channels_df.drop_duplicates(subset=['Provider/Channel'], inplace=True)

channels_df.loc[:, 'Provider/Channel'] = channels_df['Provider/Channel'].apply(match_first_part)

# print(match_first_part(string))

In [ ]:
# channels_df['Channel_Country'].isna().value_counts()
channels_df['Channel_Country'].fillna('NA', inplace=True)



It got a little nasty here because of special characters so I performed a big portion of the cleaning for networks in Excel

In [ ]:
# networks_df['Provider'].replace('Ã', 'A')
# networks_df['Provider'].replace('Ü', 'U')

replacement_dict = {'Ã': 'A', 'Ü': 'U', 'ó':'o','é':'e'}

networks_df = pd.read_csv('C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/v6_networks_cleaned.csv')

# Apply the replacements to the 'Provider' column
networks_df['Provider'] = networks_df['Provider'].replace(replacement_dict, regex=True)

networks_df


#### Cleaning Channel Instances

In [7]:
channel_instances_df = pd.read_csv('C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/v6_chan_instances_cleaned.csv')

In [8]:
channel_instances_df.head(3)

,Provider/Channel,Sattelite_Name,freq_alt,beam_alt,sr_alt,fec_alt,Compression,eirp_alt
0,AFN Prime Pacific,NSS-9,4055 L,tp GLL7,11000.0,1/2,MPEG-4/SD,34-35
1,AFN News,NSS-9,4055 L,tp GLL7,11000.0,1/2,MPEG-4/SD,34-35
2,AFN Sports,NSS-9,4055 L,tp GLL7,11000.0,1/2,MPEG-4/SD,34-35


In [ ]:
# Remove irrelevant columns
instances_df = channel_instances_df.drop(columns=['VPID','Unnamed: 0', 'encryption_alt', 'Channel','Languages','Provider_URL', 'Channel', 'Channel_URL',
       'is_Provider','Channel_Country','Provider_Country','Freq/beam', 'SR/FEC', 'undef', 'Audio', 'Encryption', 'Src_Updated','Pkg_Name', 'Pkg_URL', ])

instances_df.loc[:, 'Provider/Channel'] = instances_df['Provider/Channel'].apply(match_first_part)

# instances_df.head(0)
display_max_length(instances_df)

In [ ]:
instances_df.head(1)

# Retrieve the original Sat. name before importing to mysql
instances_df['Sattelite_Name'] = instances_df['Sattelite_Name'].apply(get_normal_sat_name)
# instances_df.drop_duplicates(subset=['Provider/Channel', 'Sattelite_Name', 'freq_alt'],inplace=True)

In [ ]:
instances_df.shape

instances_df.to_csv('C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/v6_chan_instances_cleaned_1.csv', index = False)

In [ ]:
instances_df.loc[:, 'Provider/Channel'] = instances_df['Provider/Channel'].apply(match_first_part)

In [ ]:
instance_lng_df = pd.read_csv('C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/v6_channels_instance_language.csv')

instance_lng_df.head(0)
instance_lng_df.shape

Do the same for Languages and Encryption

In [ ]:
instance_lng_df.loc[:, 'Provider/Channel'] = instance_lng_df['Provider/Channel'].apply(match_first_part)

In [ ]:
instance_lng_df.drop_duplicates(subset=['Provider/Channel', 'Sattelite_Name', 'freq_alt', 'Languages'],inplace=True)

instance_lng_df.shape

In [ ]:
instance_lng_df.to_csv('C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/v6_chan_instances_lng_cleaned.csv', index = False)

In [ ]:
instance_enc_df = pd.read_csv('C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/v6_channels_instance_encryption.csv')
instance_enc_df.shape


In [ ]:
instance_enc_df.head(0)

In [ ]:
instance_enc_df.loc[:, 'Provider/Channel'] = instance_enc_df['Provider/Channel'].apply(match_first_part)

In [ ]:
instance_enc_df.drop_duplicates(subset=['Provider/Channel', 'Sattelite_Name', 'freq_alt', 'encryption_alt'],inplace=True)

instance_enc_df.shape

In [ ]:
instance_enc_df.to_csv('C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/v6_chan_instances_enc_cleaned.csv', index = False)

When Importing to mysql, I noticed that the FK was violated because satellite names in channel_instances was in the safe format, (e.g. nss-9 not nss 9). Hence, I defined the get_normal_sat_name method and applied it below

In [ ]:
instance_lng_df = pd.read_csv('C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/v6_chan_instances_lng_cleaned.csv')


instance_lng_df['Sattelite_Name'] = instance_lng_df['Sattelite_Name'].apply(get_normal_sat_name)

instance_lng_df
instance_lng_df.to_csv('C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/v6_chan_instances_lng_cleaned_1.csv', index = False)

In [ ]:
instance_enc_df = pd.read_csv('C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/v6_chan_instances_enc_cleaned.csv')


instance_enc_df['Sattelite_Name'] = instance_enc_df['Sattelite_Name'].apply(get_normal_sat_name)

instance_enc_df
# instance_enc_df.to_csv('C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/v6_chan_instances_enc_cleaned_1.csv', index = False)

In [ ]:
instance_enc_df.dropna(subset=['encryption_alt'], inplace=True)

instance_enc_df.to_csv('C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/v6_chan_instances_enc_cleaned_1.csv', index = False)
# instance_enc_df.keys()